<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/HAFSA_Atmosphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Retrieves HAFS-A data from AWS along a storm's track using files generated by getStormTrack.ipynb.

Set up environment

In [ ]:
!pip install cfgrib

In [ ]:
from tropycal import tracks, rain
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt
import cfgrib
import numpy as np

User parameters

In [ ]:
name = "Milton"
tcNum = "14"
filepath = f"/content/{name}"
trackType = "major"

Open TC Data

In [ ]:
tc = pd.read_csv(filepath + "/hurdat2_" + name + "_" + trackType + ".csv")

In [ ]:
tc

Retrieve HAFS-A Atmospheric Data

In [ ]:
#Sample URL: https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/20241007/12/13l.2024100712.hfsa.mom6.f000.nc
#            https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/20241007/12/13l.2024100712.hfsa.parent.atm.

bucket = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/"

In [ ]:
stormMSLP = {} #Stores pressure along TC track
parentMSLP = {}

In [ ]:
dateFormat = "%Y-%m-%d %H:%M:%S"

start = tc.time[0]
startDate, startTime = start.split(" ")
startYear, startMonth, startDay = startDate.split("-")
startHour, startMinute, startSecond = startTime.split(":")

for row in tc.iloc:
  rowTime = dt.strptime(row.time, dateFormat)
  if rowTime.hour % 3 != 0 or rowTime.minute != 0: continue #Skip any lines that don't have a HAFS forecast at the same time

  tDiff = rowTime - dt.strptime(start, dateFormat)
  fHour = str(int(tDiff.total_seconds() / 3600))
  while len(fHour) < 3: fHour = "0" + fHour

  parentURL = bucket + startDate.replace("-", "") + "/" + startHour + "/" + tcNum + "l." + startDate.replace("-", "") + startHour + ".hfsa.parent.atm.f" + fHour + ".grb2"
  stormURL = bucket + startDate.replace("-", "") + "/" + startHour + "/" + tcNum + "l." + startDate.replace("-", "") + startHour + ".hfsa.storm.atm.f" + fHour + ".grb2"

  !wget -O parentData.grb2 {parentURL}
  !wget -O stormData.grb2 {stormURL}

  stormData = xr.open_dataset("stormData.grb2", engine="cfgrib", decode_timedelta=True, filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'meanSea'})
  parentData = xr.open_dataset("parentData.grb2", engine="cfgrib", decode_timedelta=True, filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'meanSea'})

  #print("\n", row.lat)
  #print(float(stormData.sel(latitude=row.lat, method="nearest").sel(longitude=row.lon, method="nearest").latitude.data))
  #print(float(parentData.sel(latitude=row.lat, method="nearest").sel(longitude=row.lon, method="nearest").latitude.data))

  #print("\n", row.lon+360)
  #print(float(stormData.sel(latitude=row.lat, method="nearest").sel(longitude=row.lon+360, method="nearest").longitude.data))
  #print(float(parentData.sel(latitude=row.lat, method="nearest").sel(longitude=row.lon+360, method="nearest").longitude.data))

  #stormMSLP[row.time] = float(stormData.sel(latitude=row.lat, method="nearest").sel(longitude=row.lon+360, method="nearest").prmsl.data)/100
  #parentMSLP[row.time] = float(parentData.sel(latitude=row.lat, method="nearest").sel(longitude=row.lon+360, method="nearest").prmsl.data)/100

  parentMSLP[row.time] = np.min(parentData.prmsl.data[np.isnan(parentData.prmsl.data)==False]) / 100 #Convert from Pa to hPa
  stormMSLP[row.time] = np.min(stormData.prmsl.data[np.isnan(stormData.prmsl.data)==False]) / 100
  print("Best:", row.mslp, "\nParent: ", parentMSLP[row.time], "\nStorm: ", stormMSLP[row.time])

In [ ]:
parentMSLP

In [ ]:
stormMSLP

Create MSLP Figure

In [ ]:
modelDates = [dt.strptime(datetime, dateFormat) for datetime in parentMSLP.keys()]
btDates = [dt.strptime(datetime, dateFormat) for datetime in tc.time]

In [ ]:
mslpFig = plt.figure(figsize=(10, 5))
mslpAx = mslpFig.add_axes([0.1,0.1,0.8,0.8])

mslpAx.plot(btDates, tc.mslp, label="BEST", color="black", linewidth=2)
mslpAx.plot(modelDates, parentMSLP.values(), label="Parent", color="green", linewidth=2)
mslpAx.plot(modelDates, stormMSLP.values(), label="Storm", color="purple", linewidth=2)

mslpAx.set_xlabel("Time")
mslpAx.set_ylabel("MSLP (hPa)")
mslpAx.grid(alpha=0.5)
mslpAx.legend()

mslpFig.savefig(filepath + "/mslp.png")

Calculate RMSE